In [1]:
from __future__ import division
from pyomo.environ import *
import numpy as np
import pandas as pd

In [56]:
# base_price=[[10,10,10],
#             [10,10,10]]

# intercept = [23,20]
# coef_1 =[-6,-5]
# coef_2 =[0.01,0.02]
# coef_3 = [1,1.2]
# coef_4 = [-0.003,-0.004]
products=2
coef_p1 = [-6,0.01,1,-0.003,23]
coef_p2 = [1.2,-0.004,-5,0.02,20]
coef = [coef_p1,coef_p2]

base_price_1=[10,10,10]
base_price_2=[10,10,10]

edlp_p1=[9.5,10,10]
edlp_p2 =[10,10,9.6]
tpr_p1=[0,5,15]
tpr_p2 =[15,5,0]
flag_p1=[1,0,0]
flag_p2=[0,0,1]

target_trade_spend = [276860,223573]
target_edlp_spend = [63366,35525]
target_tpr_spend = [213494,188048]

idx = range(6)
products = 2
weeks = 3
tot = products*weeks

In [52]:
def cal_sales(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef):
    
    unit_sales = []
    for i in  range(weeks):
#         print(edlp_p1[i],coef[0],flag_p1[i])
        a = (log( edlp_p1[i])*coef[0])* flag_p1[i]
        b = (log(base_price_1[i])*coef[0]+ tpr_p1[i]*coef[1])*(1- flag_p1[i])
        
        c = (log( edlp_p2[i])*coef[2])* flag_p2[i]
        d = (log(base_price_2[i])*coef[2]+ tpr_p2[i]*coef[3])*(1- flag_p2[i])
        
#         print(a,' $ ',b,' $ ',c,' $ ',d,' $ ',coef[4])
        unit_sales.append(exp(a+b+c+d+coef[4]))                 
   
    return unit_sales

def calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,j):
    price = []
    edlp = [edlp_p1,edlp_p2]
    tpr = [tpr_p1,tpr_p2]
    flag = [flag_p1,flag_p2]
    base_price=[base_price_1,base_price_2]
    for i in range(weeks):
        price.append((base_price[j][i]*(1-tpr[j][i]/100)*(1-flag[j][i]))  +  (edlp[j][i]*flag[j][i]))
                   
    return price
                   
                   
def calc_dollar_sales(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef):
    dollar_sales=[]
    for j in range(products):
        unit_sales= cal_sales(edlp_p1, tpr_p1, flag_p1, 
                              edlp_p2, tpr_p2, flag_p2, coef[j])
        price = calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,j)

        dollar_sales_single = [unit_sales[i]*price[i] for i in range(weeks)]
        dollar_sales.append(dollar_sales_single)
    
    return dollar_sales

def calc_total_trade_spent(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef):
    total_trade_spent = []
    for j in range(products):
        unit_sales= cal_sales(edlp_p1, tpr_p1, flag_p1, 
                              edlp_p2, tpr_p2, flag_p2, coef[j])
        price = calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,j)

        trade_spent = [base_price[j][i]-price[i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[i] for i in range(weeks)]
        total_trade_spent.append(total_trade_spent_partial)
        
    return total_trade_spent

def calc_edlp_trade_spent(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef):
    edlp_trade_spent = []
    flag = [flag_p1,flag_p2]
    for j in range(products):
        unit_sales= cal_sales(edlp_p1, tpr_p1, flag_p1, 
                              edlp_p2, tpr_p2, flag_p2, coef[j])
        price = calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,j)

        trade_spent = [base_price[j][i]-price[i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[i] for i in range(weeks)]

        edlp_trade_spent_partial = [total_trade_spent_partial[i]*flag[j][i] for i in range(weeks)]
        edlp_trade_spent.append(edlp_trade_spent_partial)
    return edlp_trade_spent


In [57]:
def create_model(init):
    print("Init value= ",init)
    def initial(model,i):
        return init[i]

    model = ConcreteModel(name='Spend_Optim')
    
    model.edlp_p1 = Var(range(weeks), initialize=10, bounds=(9.5, 10), domain=NonNegativeReals)
    model.tpr_p1 = Var(range(weeks), initialize=5, bounds=(5, 50), domain=NonNegativeReals)
    model.flag_p1 = Var(range(weeks),initialize=initial,domain=Binary)
    
    model.edlp_p2 = Var(range(weeks), initialize=10, bounds=(9.5, 10), domain=NonNegativeReals)
    model.tpr_p2 = Var(range(weeks), initialize=5, bounds=(5, 50), domain=NonNegativeReals)
    model.flag_p2 = Var(range(weeks),initialize=initial,domain=Binary)
    
    dollar_sales = calc_dollar_sales(model.edlp_p1, model.tpr_p1, model.flag_p1,model.edlp_p2, model.tpr_p2, model.flag_p2,coef)
    total_trade_spent = calc_total_trade_spent(model.edlp_p1, model.tpr_p1, model.flag_p1,model.edlp_p2, model.tpr_p2, model.flag_p2,coef)
    edlp_trade_spent = calc_edlp_trade_spent(model.edlp_p1, model.tpr_p1, model.flag_p1,model.edlp_p2, model.tpr_p2, model.flag_p2,coef)
    
    model.obj = Objective(expr=sum(dollar_sales[0])+sum(dollar_sales[1]) ,sense= maximize)
    model.c1 = Constraint(expr = sum(total_trade_spent[0]) == target_trade_spend[0])
    model.c2 = Constraint(expr = sum(total_trade_spent[1]) == target_trade_spend[1])
    model.c3 = Constraint(expr = sum(edlp_trade_spent[0]) == target_edlp_spend[0] )
    model.c4 = Constraint(expr = sum(edlp_trade_spent[1]) == target_edlp_spend[1] )
#     model.c5 = Constraint(expr = sum(calc_flag_util(model.flag))==0 )
    return model

In [58]:
def call_solver(init, name='bonmin'):
    assert type(init) == list
    model = create_model(init)
    opt_1 = SolverFactory(name)
    results_1 = opt_1.solve(model)
    model.display()
#     print(results_1)
#     print(get_info_from_results(results_1, 'Time: '))

combinations=[[0,0,0,0,1,1]]    
for c in combinations:
    call_solver(c, 'bonmin')
    break

Init value=  [0, 0, 0, 0, 1, 1]
Model Spend_Optim

  Variables:
    edlp_p1 : Size=3, Index=edlp_p1_index
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
          0 :   9.5 : 9.754932949990252 :    10 : False : False : NonNegativeReals
          1 :   9.5 : 9.501569774312637 :    10 : False : False : NonNegativeReals
          2 :   9.5 : 9.754932949990252 :    10 : False : False : NonNegativeReals
    tpr_p1 : Size=3, Index=tpr_p1_index
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :     5 : 10.203736454217927 :    50 : False : False : NonNegativeReals
          1 :     5 :   27.1454470698727 :    50 : False : False : NonNegativeReals
          2 :     5 :  10.20373645421793 :    50 : False : False : NonNegativeReals
    flag_p1 : Size=3, Index=flag_p1_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.0 :     1 : False : False : Binary
          1 :     0 :   1.0 :     1 : Fals

In [50]:
def cal_sales(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef):
    
    unit_sales = []
    for i in  range(weeks):
#         print(edlp_p1[i],coef[0],flag_p1[i])
        a = (np.log( edlp_p1[i])*coef[0])* flag_p1[i]
        b = (np.log(base_price_1[i])*coef[0]+ tpr_p1[i]*coef[1])*(1- flag_p1[i])
        
        c = (np.log( edlp_p2[i])*coef[2])* flag_p2[i]
        d = (np.log(base_price_2[i])*coef[2]+ tpr_p2[i]*coef[3])*(1- flag_p2[i])
        
#         print(a,' $ ',b,' $ ',c,' $ ',d,' $ ',coef[4])
        unit_sales.append(np.exp(a+b+c+d+coef[4]))                 
   
    return unit_sales

def calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,j):
    price = []
    edlp = [edlp_p1,edlp_p2]
    tpr = [tpr_p1,tpr_p2]
    flag = [flag_p1,flag_p2]
    base_price=[base_price_1,base_price_2]
    for i in range(weeks):
        price.append((base_price[j][i]*(1-tpr[j][i]/100)*(1-flag[j][i]))  +  (edlp[j][i]*flag[j][i]))
                   
    return price
                   
                   
def calc_dollar_sales(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef):
    dollar_sales=[]
    for j in range(products):
        unit_sales= cal_sales(edlp_p1, tpr_p1, flag_p1, 
                              edlp_p2, tpr_p2, flag_p2, coef[j])
        price = calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,j)

        dollar_sales_single = [unit_sales[i]*price[i] for i in range(weeks)]
        dollar_sales.append(dollar_sales_single)
    
    return dollar_sales

def calc_total_trade_spent(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef):
    total_trade_spent = []
    for j in range(products):
        unit_sales= cal_sales(edlp_p1, tpr_p1, flag_p1, 
                              edlp_p2, tpr_p2, flag_p2, coef[j])
        price = calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,j)

        trade_spent = [base_price[j][i]-price[i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[i] for i in range(weeks)]
        total_trade_spent.append(total_trade_spent_partial)
        
    return total_trade_spent

def calc_edlp_trade_spent(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef):
    edlp_trade_spent = []
    flag = [flag_p1,flag_p2]
    for j in range(products):
        unit_sales= cal_sales(edlp_p1, tpr_p1, flag_p1, 
                              edlp_p2, tpr_p2, flag_p2, coef[j])
        price = calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,j)

        trade_spent = [base_price[j][i]-price[i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[i] for i in range(weeks)]

        edlp_trade_spent_partial = [total_trade_spent_partial[i]*flag[j][i] for i in range(weeks)]
        edlp_trade_spent.append(edlp_trade_spent_partial)
    return edlp_trade_spent


In [51]:
products=2
coef_p1 = [-6,0.01,1,-0.003,23]
coef_p2 = [1.2,-0.004,-5,0.02,20]
coef = [coef_p1,coef_p2]

base_price_1=[10,10,10]
base_price_2=[10,10,10]

edlp_p1=[9.5,10,10]
edlp_p2 =[10,10,9.6]
tpr_p1=[0,5,15]
tpr_p2 =[15,5,0]
flag_p1=[1,0,0]
flag_p2=[0,0,1]

# cal_sales(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef[0]),cal_sales(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef[1])
# calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,0),calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,1)
# calc_dollar_sales(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef)
# calc_total_trade_spent(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef)
# calc_edlp_trade_spent(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef)

[[63366.278926377876, 0.0, 0.0], [0.0, 0.0, 35525.139769282396]]